In [1]:
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI
from llama_index.core.agent.workflow import AgentWorkflow
from llama_index.core.tools import FunctionTool
import os, chromadb
from dotenv import find_dotenv, load_dotenv
from llama_index.core.workflow import Context
from llama_index.core import VectorStoreIndex
from llama_index.core.tools import QueryEngineTool
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.chroma import ChromaVectorStore

c:\Users\kevin\miniconda3\envs\agents_mcp_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv(find_dotenv())
hf_token = os.getenv("HF_TOKEN")

---

#### Initialising Agents

In [3]:
# define sample Tool -- type annotations, function names, and docstrings, are all included in parsed schemas!
def multiply(a: int, b: int) -> int:
    """Multiplies two integers and returns the resulting integer"""
    return a * b

In [17]:
# initialize llm
llm = HuggingFaceInferenceAPI(model_name="Qwen/Qwen2.5-Coder-32B-Instruct", temperature=0.7, max_tokens=1000, 
                              token=hf_token, provider="auto")

In [18]:
# initialize agent
agent = AgentWorkflow.from_tools_or_functions(
    tools_or_functions = [FunctionTool.from_defaults(multiply)],
    llm = llm
)
agent

#### Stateless

In [6]:
response_1 = await agent.run("What is 2 times 2?")
response_2 = await agent.run("What is the result again?")

In [7]:
print(response_1.response)
print(response_1.tool_calls)
print(response_2.response)
print(response_2.tool_calls)

assistant: 2 times 2 is 4.
[ToolCallResult(tool_name='multiply', tool_kwargs={'a': 2, 'b': 2}, tool_id='5d228ecb-ad5d-4202-830d-21d811ab099f', tool_output=ToolOutput(blocks=[TextBlock(block_type='text', text='4')], tool_name='multiply', raw_input={'args': (), 'kwargs': {'a': 2, 'b': 2}}, raw_output=4, is_error=False), return_direct=False)]
assistant: I cannot answer the question with the provided tools.
[]


#### Remembering State

In [19]:
ctx = Context(agent)

In [ ]:
response_3 = await agent.run("What is 2 times 27?", ctx=ctx)
response_4 = await agent.run("What is the result again?", ctx=ctx)

In [28]:
print(response_3.response)
print(response_3.tool_calls)
print(response_4.response)
print(response_4.tool_calls)

assistant: 2 times 27 is 54.
[ToolCallResult(tool_name='multiply', tool_kwargs={'a': 2, 'b': 27}, tool_id='42e23c6e-a392-40f2-bc4b-7157586700f7', tool_output=ToolOutput(blocks=[TextBlock(block_type='text', text='54')], tool_name='multiply', raw_input={'args': (), 'kwargs': {'a': 2, 'b': 27}}, raw_output=54, is_error=False), return_direct=False)]
assistant: The result of 2 times 27 is 54.
[]


---

#### RAG Agents with QueryEngineTools

In [12]:
embed_model = HuggingFaceEmbedding("BAAI/bge-small-en-v1.5")

db = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db.get_or_create_collection("nlp_papers")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

index = VectorStoreIndex.from_vector_store(vector_store, embed_model=embed_model)

llm = HuggingFaceInferenceAPI(
    model_name="Qwen/Qwen2.5-Coder-32B-Instruct",
    temperature=0.7,
    max_tokens=1000,
    token=hf_token,
    provider="auto"
)
query_engine = index.as_query_engine(llm=llm)

In [13]:
query_engine_tool = QueryEngineTool.from_defaults(query_engine, name="my_nlp_research_tool", 
                                                  description="Useful for getting information about Kevin's NLP research", 
                                                  return_direct=False)

In [14]:
query_engine_agent = AgentWorkflow.from_tools_or_functions(
    tools_or_functions=[query_engine_tool],
    llm=llm,
    system_prompt="You are a helpful assistant that has access to a database containing NLP research results for Kevin. "
)

In [ ]:
response = await query_engine_agent.run("What is the main contribution of Kevin Martinez to NLP?")
print(response.response)
print(response.tool_calls)

assistant: Kevin Martinez's main contribution to NLP is his work on developing and evaluating methods for detecting fake news in Spanish. This includes preprocessing text data, comparing various text representation techniques, and assessing the performance of different machine learning and deep learning models for this specific task.
[ToolCallResult(tool_name='my_nlp_research_tool', tool_kwargs={'input': 'What is the main contribution of Kevin Martinez to NLP?'}, tool_id='4521015d-24cf-442f-8509-656eb2fbd698', tool_output=ToolOutput(blocks=[TextBlock(block_type='text', text="Kevin Martinez's main contribution, as detailed in the provided context, pertains to the development and evaluation of methods for detecting fake news in Spanish. Specifically, the work involves preprocessing text data through normalization, stop-word removal, stemming, tokenization, and padding, as well as comparing different text representation techniques such as Bag of Words (BoW), Term Frequency-Inverse Documen

In [15]:
response = await query_engine_agent.run("What is the main contribution of Geoffrey Hinton to NLP?")
print(response.response)
print(response.tool_calls)

assistant: Geoffrey Hinton is well-known for his work on artificial neural networks, backpropagation, and deep learning. His contributions to the development of deep neural networks have been instrumental, including the revival of interest in backpropagation and the creation of models like Boltzmann machines and recurrent neural networks. While his most famous work might not be directly in NLP, his advancements in neural networks have significantly impacted NLP, enabling more effective language models and natural language processing systems.
[ToolCallResult(tool_name='my_nlp_research_tool', tool_kwargs={'input': 'What is the main contribution of Geoffrey Hinton to NLP?'}, tool_id='e983b33a-a5dc-45fc-ace1-75376e953f9b', tool_output=ToolOutput(blocks=[TextBlock(block_type='text', text="The provided context does not contain any information about Geoffrey Hinton's contributions to NLP. Therefore, based solely on the given information, it is not possible to answer the query regarding Geoffr

---